In [27]:
import pandas as pd

df = pd.read_csv("cleaned_lyrics_data.csv")
df

,artist,song,lyrics,complexity,swear_words_ratio
0,gojira,the cell,"All this evil, sanctified You're not alone You...",8.8,0.000000
1,gojira,04,hello wooboot... this is wooboot... hum..... y...,3.1,0.025000
2,gojira,the way of all flesh,"Anything that has a shape will crumble away, d...",6.4,0.008333
3,gojira,to sirius,NaN,7.8,0.000000
4,gojira,on the bota,"Lost, a lion came and said: ""Come, on the brin...",3.1,0.022727
5,gojira,the silver cord,INSTRUMENTAL,7.8,0.031915
6,gojira,wolf down the earth,First of all you will eat all the bodies No ma...,7.8,0.000000
7,gojira,from mars,I just looked myself Straight in the eyes and ...,3.1,0.000000
8,gojira,death of me,A giant bright eagle has come on me and taken ...,8.3,0.013761
9,gojira,the link,"I want to protect myself from others Dream, I ...",5.3,0.020661


In [58]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Spectral10
output_notebook()

top10comp = df.nlargest(10, "complexity")
top10swears = df.nlargest(10, "swear_words_ratio")
top10comp["colors"] = Spectral10



p = figure(plot_height = 800, plot_width = 1500, x_range = list(top10comp["song"]), title = "Top 10 Complexity Socres by Song Name")
p.vbar("song", 0.5, "complexity", source = top10comp, color = "colors")
show(p)

Loading BokehJS ...